In [3]:
import duckdb
import pandas as pd

con = duckdb.connect('db_local.duckdb2')
caminho_pasta = './dados-parquet/*.parquet'

#query = f"""DROP TABLE IF EXISTS teste; CREATE TABLE teste AS SELECT * FROM '{ caminho_pasta }' """
#sql = con.execute(query)
#sql.fetch_df()



In [4]:
query = f"""
    DROP TABLE IF EXISTS teste1;
    CREATE TABLE teste1 
    AS
    SELECT 
    DISTINCT 
        NM_INDICADOR_NEGOCIO
        , NM_PERFIL_CLIENTE
        ,NM_TIPO_CLASSE_SOCIAL, 
    COUNT(*) AS QTD
    FROM '{ caminho_pasta }'
    WHERE NM_INDICADOR_NEGOCIO = 'Instalação'
    GROUP BY 
    NM_INDICADOR_NEGOCIO,
    NM_PERFIL_CLIENTE,
    NM_TIPO_CLASSE_SOCIAL
 """
#PROSPECT E MIGRAÇÃO E IGUAL NOVO'

con.execute(query).df()



,Count
0,21


In [ ]:
import plotly as px

con.execute('select * from duckdb_tables()').df()

Q = '''
    SELECT * FROM teste1
    '''

teste1 = con.execute(Q).df()

display(teste1['NM_PERFIL_CLIENTE'].unique())

Q = '''
    SELECT * FROM teste1
    WHERE 1=1 NM_PERFIL_CLIENTE = 'Migração - NET' OR NM_PERFIL_CLIENTE = 'Migração - NET'
    ORDER BY QTD DESC
    '''
teste1 = con.execute(Q).df()

colunas = ["NM_INDICADOR_NEGOCIO", "NM_PERFIL_CLIENTE", "NM_TIPO_CLASSE_SOCIAL", "QTD"]

df = pd.DataFrame(teste1, columns=colunas)

fig = px.hist_series(
    df,
    x="NM_PERFIL_CLIENTE",
    y="QTD",
    color="NM_TIPO_CLASSE_SOCIAL",
    title="Quantidade por Perfil de Cliente e Classe Social (Instalação)",
    labels={
        "NM_PERFIL_CLIENTE": "Perfil do Cliente",
        "QTD": "Quantidade",
        "NM_TIPO_CLASSE_SOCIAL": "Classe Social", 
    },
    barmode="group"
)
fig.show()


array(['Migração - NET', 'Prospect', 'Base'], dtype=object)

In [36]:
query = f"""
SELECT DISTINCT NM_INDICADOR_NEGOCIO, NM_PERFIL_CLIENTE, NM_TIPO_CLASSE_SOCIAL
FROM read_parquet('{caminho_pasta}')
ORDER BY NM_INDICADOR_NEGOCIO DESC
"""
resultado = con.execute(query).fetch_df()
display(resultado)

,NM_INDICADOR_NEGOCIO,NM_PERFIL_CLIENTE,NM_TIPO_CLASSE_SOCIAL
0,Venda Bruta,Migração - NET,B2
1,Venda Bruta,Prospect,D/E
2,Venda Bruta,Migração - NET,B1
3,Venda Bruta,Prospect,Não Informado
4,Venda Bruta,Base,Não Informado
...,...,...,...
58,Desistência,Migração - NET,A
59,Desistência,Prospect,A
60,Desistência,Base,C1
61,Desistência,Base,C2


In [69]:
'''query = f"""
        SELECT NM_TIPO_CLASSE_SOCIAL
        , COUNT(*) AS quantidade
        FROM read_parquet('{caminho_pasta}')
        GROUP BY NM_TIPO_CLASSE_SOCIAL
        ORDER BY quantidade DESC
        """
resultado = con.execute(query).fetch_df()
display(resultado)'''

query = f"""
        SELECT NM_TIPO_CLASSE_SOCIAL
        FROM read_parquet('{caminho_pasta}')
        """
resultadoNM_TIPO_CLASSE_SOCIAL = con.execute(query).fetch_df()
display(resultadoNM_TIPO_CLASSE_SOCIAL)

resultadojson = resultado.to_json(orient="records", force_ascii=False)

print((resultadojson))

,NM_TIPO_CLASSE_SOCIAL
0,B2
1,B2
2,B2
3,D/E
4,Não Informado
...,...
4467582,Não Informado
4467583,Não Informado
4467584,B2
4467585,A


[{"NM_TIPO_CLASSE_SOCIAL":"Não Informado","quantidade":1351281},{"NM_TIPO_CLASSE_SOCIAL":"B2","quantidade":1219828},{"NM_TIPO_CLASSE_SOCIAL":"C1","quantidade":683928},{"NM_TIPO_CLASSE_SOCIAL":"B1","quantidade":496942},{"NM_TIPO_CLASSE_SOCIAL":"C2","quantidade":331922},{"NM_TIPO_CLASSE_SOCIAL":"A","quantidade":300721},{"NM_TIPO_CLASSE_SOCIAL":"D\/E","quantidade":82965}]


In [49]:
import duckdb

caminho_pasta = './dados-parquet/*.parquet'

colunas = [
    "NM_INDICADOR_NEGOCIO",
    "NM_PERFIL_CLIENTE",
    "NM_TIPO_CLASSE_SOCIAL"
]


con = duckdb.connect()

def consultar_distintos_e_contar(colunas, caminho_pasta):
    for coluna in colunas:
        query = f"""
        SELECT {coluna}, COUNT(*) AS quantidade
        FROM read_parquet('{caminho_pasta}')
        GROUP BY {coluna}
        ORDER BY quantidade DESC
        """
        resultado = con.execute(query).fetch_df()
        print(f"\nContagem de valores distintos para: {coluna}")
        print(resultado)

consultar_distintos_e_contar(colunas, caminho_pasta)


import pandas as pd
import plotly.express as px

# Dados fornecidos
nm_indicador_negocio = pd.DataFrame({
    "NM_INDICADOR_NEGOCIO": ["Venda Bruta", "Instalação", "Desistência"],
    "quantidade": [2239467, 1794437, 433683]
})

nm_perfil_cliente = pd.DataFrame({
    "NM_PERFIL_CLIENTE": ["Prospect", "Base", "Migração - NET"],
    "quantidade": [4257838, 190724, 19025]
})

nm_tipo_classe_social = pd.DataFrame({
    "NM_TIPO_CLASSE_SOCIAL": [
        "Não Informado", "B2", "C1", "B1", "C2", "A", "D/E"
    ],
    "quantidade": [1351281, 1219828, 683928, 496942, 331922, 300721, 82965]
})

# Gráfico 1: NM_INDICADOR_NEGOCIO
fig1 = px.bar(nm_indicador_negocio, x="NM_INDICADOR_NEGOCIO", y="quantidade",
              title="Contagem por Indicador de Negócio",
              labels={"NM_INDICADOR_NEGOCIO": "Indicador de Negócio", "quantidade": "Quantidade"})
fig1.show()

# Gráfico 2: NM_PERFIL_CLIENTE
fig2 = px.bar(nm_perfil_cliente, x="NM_PERFIL_CLIENTE", y="quantidade",
              title="Contagem por Perfil de Cliente",
              labels={"NM_PERFIL_CLIENTE": "Perfil de Cliente", "quantidade": "Quantidade"})
fig2.show()

# Gráfico 3: NM_TIPO_CLASSE_SOCIAL
fig3 = px.bar(nm_tipo_classe_social, x="NM_TIPO_CLASSE_SOCIAL", y="quantidade",
              title="Contagem por Classe Social",
              labels={"NM_TIPO_CLASSE_SOCIAL": "Classe Social", "quantidade": "Quantidade"})
fig3.show()



Contagem de valores distintos para: NM_INDICADOR_NEGOCIO
  NM_INDICADOR_NEGOCIO  quantidade
0          Venda Bruta     2239467
1           Instalação     1794437
2          Desistência      433683

Contagem de valores distintos para: NM_PERFIL_CLIENTE
  NM_PERFIL_CLIENTE  quantidade
0          Prospect     4257838
1              Base      190724
2    Migração - NET       19025

Contagem de valores distintos para: NM_TIPO_CLASSE_SOCIAL
  NM_TIPO_CLASSE_SOCIAL  quantidade
0         Não Informado     1351281
1                    B2     1219828
2                    C1      683928
3                    B1      496942
4                    C2      331922
5                     A      300721
6                   D/E       82965


In [37]:
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
import numpy as np

# Dados combinados
dados = pd.DataFrame({
    "categoria": ["Venda Bruta", "Instalação", "Desistência",
                  "Prospect", "Base", "Migração - NET",
                  "Não Informado", "B2", "C1", "B1", "C2", "A", "D/E"],
    "x": list(range(13)),  # Índice fictício para cada categoria
    "y": [2239467, 1794437, 433683,
          4257838, 190724, 19025,
          1351281, 1219828, 683928, 496942, 331922, 300721, 82965]
})

# Regressão linear
X = dados[["x"]]
y = dados["y"]
modelo = LinearRegression()
modelo.fit(X, y)

# Previsões
dados["y_predito"] = modelo.predict(X)

# Gráfico
fig = px.scatter(dados, x="x", y="y", text="categoria",
                 title="Regressão Linear sobre Contagens por Categoria",
                 labels={"x": "Índice da Categoria", "y": "Quantidade"})
fig.add_traces(px.line(dados, x="x", y="y_predito").data)
fig.show()
